In [1]:
%load_ext autoreload
%autoreload

import os
import pandas as pd
import numpy as np
import asyncio
#from functools import partial
import twitterscraper as tws
from IPython.core.debugger import set_trace
from tqdm import tqdm_notebook
# from konlpy.tag import Okt
import texcrapy
import json
# from dask import compute, delayed
# import dask.threaded
# import dask.multiprocessing
#from dask.diagnostics import ProgressBar

#pbar = ProgressBar()
#pbar.register()
#pbar.unregister()

In [3]:
df = pd.read_excel('keywords and logos.xlsx', sheet_name='20190215')[['shortname','kw_supporter','kw_supported','keywords']]; df
_or = lambda kw: ' OR '.join(['#' + k.strip() for k in kw.split(',')])

qry_base = {row.shortname:_or(row.keywords) for row in df.itertuples()}
supporters = df.shortname[df.kw_supporter==True]
qry_sup = ' OR '.join([qry_base[sup] for sup in supporters]); qry_sup
qry = {row.shortname: '(' + qry_base[row.shortname] + ') AND (' + qry_sup + ')' if row.kw_supported==True else qry_base[row.shortname] for row in df.itertuples()}

In [3]:
qry

{'ootd': '#dailylook OR #outfit OR #bestofstreetwear OR #beststreetoutfit OR #ootd OR #outfitoftheday OR #데일리룩 OR #오오티디 OR #아웃핏',
 'category': '#designer OR #streetwear OR #luxury OR #casual OR #sportswear OR #디자이너 OR #길거리패션 OR #럭셔리 OR #캐주얼 OR #캐쥬얼 OR #스포츠웨어 OR #스포츠패션',
 'fashion': '#fashion OR #brand OR #fashionbrand OR #shoe OR #shoes OR #clothes OR #wear OR #apparel OR #패션 OR #브랜드 OR #패션브랜드 OR #옷 OR #의류 OR #슈즈 OR #신발',
 '8seconds': '#8seconds OR #8second OR #에잇세컨즈 OR #8세컨즈',
 'adidas': '#adidas OR #아디다스',
 'adidasoriginal': '#adidasoriginal OR #아디다스오리지널스 OR #아디다스오리지널 OR #아디다스오리지날스 OR #아디다스오리지날',
 'asics': '#asics OR #아식스',
 'balenciaga': '#balenciaga OR #balensiaga OR #발렌시아가',
 'bally': '#bally OR #발리',
 'beanpole': '#beanpole OR #빈폴',
 'benetton': '#benetton OR #베네통',
 'blackyak': '#blackyak OR #블랙야크',
 'buckaroo': '#buckaroo OR #버커루',
 'burberry': '#burberry OR #버버리',
 'calvinklein': '#calvinklein OR #케빈클라인 OR #켈빈클라인 OR #캘빈클라인',
 'canadagoose': '#canadagoose OR #캐나다구스',
 'chanel':

In [6]:
# 요게 필요한가?
with open('twitter_data/qry.json', 'w') as f:
    json.dump(qry, f)

In [17]:
%%time
what = ['id', 'timestamp', 'text']
texcrapy.scrap(qry, what=what, lang='ko', end='2019-01-31', download_to='scrapped/twitter')

INFO: queries: ['#dailylook OR #outfit OR #bestofstreetwear OR #beststreetoutfit OR #ootd OR #outfitoftheday OR #데일리룩 OR #오오티디 OR #아웃핏 since:2010-01-01 until:2010-06-15', '#dailylook OR #outfit OR #bestofstreetwear OR #beststreetoutfit OR #ootd OR #outfitoftheday OR #데일리룩 OR #오오티디 OR #아웃핏 since:2010-06-15 until:2010-11-28', '#dailylook OR #outfit OR #bestofstreetwear OR #beststreetoutfit OR #ootd OR #outfitoftheday OR #데일리룩 OR #오오티디 OR #아웃핏 since:2010-11-28 until:2011-05-13', '#dailylook OR #outfit OR #bestofstreetwear OR #beststreetoutfit OR #ootd OR #outfitoftheday OR #데일리룩 OR #오오티디 OR #아웃핏 since:2011-05-13 until:2011-10-26', '#dailylook OR #outfit OR #bestofstreetwear OR #beststreetoutfit OR #ootd OR #outfitoftheday OR #데일리룩 OR #오오티디 OR #아웃핏 since:2011-10-26 until:2012-04-09', '#dailylook OR #outfit OR #bestofstreetwear OR #beststreetoutfit OR #ootd OR #outfitoftheday OR #데일리룩 OR #오오티디 OR #아웃핏 since:2012-04-09 until:2012-09-22', '#dailylook OR #outfit OR #bestofstreetwear OR #bestst


Wall time: 5h 1min 17s


In [5]:
what = ['id', 'timestamp', 'text']
values = [delayed(scrap)(bname, q, what=what, lang='ko', end='2019-01-31', download_to='twitter_data') for bname,q in list(qry.items())[:]]

In [6]:
%%time
results = compute(*values, scheduler='threads')

INFO: queries: ['#placesplusfaces OR #플레이시스플러스페이시스 since:2010-01-01 until:2010-06-15', '#placesplusfaces OR #플레이시스플러스페이시스 since:2010-06-15 until:2010-11-28', '#placesplusfaces OR #플레이시스플러스페이시스 since:2010-11-28 until:2011-05-13', '#placesplusfaces OR #플레이시스플러스페이시스 since:2011-05-13 until:2011-10-26', '#placesplusfaces OR #플레이시스플러스페이시스 since:2011-10-26 until:2012-04-09', '#placesplusfaces OR #플레이시스플러스페이시스 since:2012-04-09 until:2012-09-22', '#placesplusfaces OR #플레이시스플러스페이시스 since:2012-09-22 until:2013-03-06', '#placesplusfaces OR #플레이시스플러스페이시스 since:2013-03-06 until:2013-08-19', '#placesplusfaces OR #플레이시스플러스페이시스 since:2013-08-19 until:2014-02-01', '#placesplusfaces OR #플레이시스플러스페이시스 since:2014-02-01 until:2014-07-17', '#placesplusfaces OR #플레이시스플러스페이시스 since:2014-07-17 until:2014-12-30', '#placesplusfaces OR #플레이시스플러스페이시스 since:2014-12-30 until:2015-06-14', '#placesplusfaces OR #플레이시스플러스페이시스 since:2015-06-14 until:2015-11-27', '#placesplusfaces OR #플레이시스플러스페이시스 since:2015-11-27 until:201

Wall time: 3h 46min 2s
